In [1]:
import pyrosetta
pyrosetta.distributed.maybe_init(**{
    "options": "-corrections::beta_nov16 true",
    "extra_options":{
        "-out:level": "100",
}})

In [2]:
from glob import glob

for design_fname in glob('/home/broerman/projects/CSD/round_2/disulfides/staples/*Y_*.pdb'):
    design_name = design_fname.split('/')[-1]
    if 'AY_' in design_name:
        partner_fname = '/home/broerman/projects/CSD/round_2/af2/split_Y/' + design_name[:-13].replace('AY_', 'BY_') + '.pdb'
        pose_Y = pyrosetta.pose_from_pdb(design_fname)
        pose_B = pyrosetta.pose_from_pdb(partner_fname)
    elif 'BY_' in design_name:
        partner_fname = '/home/broerman/projects/CSD/round_2/af2/split_Y/' + design_name[:-13].replace('BY_', 'AY_') + '.pdb'
        pose_Y = pyrosetta.pose_from_pdb(partner_fname)
        pose_B = pyrosetta.pose_from_pdb(design_fname)
    pyrosetta.rosetta.core.pose.append_pose_to_pose(pose_Y, pose_B, True)
    pose_Y.dump_pdb(f'/home/broerman/projects/CSD/round_2/disulfides/staples/combine_Y/{design_name[:5]}AB_{design_name[5:]}')

In [3]:
# alphafold

from Bio import SeqIO
from collections import defaultdict
from glob import glob
from IPython.utils.io import capture_output

files_by_len = defaultdict(list)

design_fnames = glob( '/home/broerman/projects/CSD/round_2/disulfides/staples/*A_*.pdb') + \
                glob('/home/broerman/projects/CSD/round_2/disulfides/staples/*B_*.pdb') + \
                glob('/home/broerman/projects/CSD/round_2/disulfides/staples/combine_Y/*.pdb')

for design_fname in design_fnames:

    sequence = ''
    with capture_output() as captured: # otherwise memory overloads lol
        for i, record in enumerate(SeqIO.parse(design_fname, 'pdb-atom')):
            sequence += str(record.seq)
    
    files_by_len[len(sequence)].append(design_fname)

task_lines = []

for length, file_list in files_by_len.items():
    file_str = ' '.join(file_list)
    task_lines.append(f"/home/rdkibler/software/alphafold/superfold {file_str} --models all --max_recycles 4 --initial_guess --simple_rmsd --out_dir /home/broerman/projects/CSD/round_2/disulfides/af2_models\n")

task_lines.sort(key=lambda x: -len(x))

with open('af2_disulfides.tasks', 'w') as f:
    f.write(''.join(task_lines))

In [4]:
print('sbatch -a 1-$(cat af2_disulfides.tasks | wc -l) af2_disulfides.sh')

sbatch -a 1-$(cat af2_CSDs.tasks | wc -l) af2_CSDs.sh
